In [ ]:
# install pygad and dependencies (ommit index-url to install cpu version of pytorch)
# also supports keras for GANN 
%pip install numpy
%pip install matplotlib
%pip install torch  --index-url https://download.pytorch.org/whl/cu121
%pip install pygad

In [2]:
import pygad, pygad.gann, pygad.nn, torch
import numpy as np

Vind een matrix met getallen tussen -2 eb 5, wanneer je die elementwijs vermenigvuldigd met de matrix [4,-2,3.5,5,-11,-4.7] een som heeft gelijk is aan 44.

In [2]:
input_values= np.array( [4,-2,3.5,5,-11,-4.7] )
desired_output = 44

# bereken de fitness -> hoe groter hoe beter.
def fitness_func(ga_instance:pygad.GA, solution:np.ndarray, solution_idx:int):
    output = np.sum(solution*input_values)
    fitness = 1.0 / np.abs(output - desired_output) # 1 / Absolute Error
    return fitness


In [3]:
fitness_function = fitness_func

num_generations = 50
num_parents_mating = 4

sol_per_pop = 8                     # populatie grootte
num_genes = len(input_values)       # == 6

init_range_low = -2                 # initiele genen zijn een random waarde tussen -2 en 5
init_range_high = 5

gene_space = {"low": -2, "high": 5} # minimum en maximum waarde voor een gen, kan ook een discrete lijst zijn.  

parent_selection_type = "sss"       # elitism -> Steady State Selection, beste parents gaan over naar de volgende generatie 
keep_parents = 1                    # aantal ouderparen dat overgaan naar volgende generatie 

crossover_type = "single_point"     # single_point = oudergenen op 1 plaats gespltst en samengevoegd
crossover_probability = 0.5         # 50% kans dat genen splitsten worden, anders gen van 1 ouder. 

mutation_type = "random"       
mutation_percent_genes = 25         # 25% kans op mutatie

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       gene_space= gene_space,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       crossover_probability=crossover_probability,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes)

ga_instance.run()
solution, fitness, _ = ga_instance.best_solution()
f"De beste oplossing {solution} met een fitness van {fitness:0.4f}"

'De beste oplossing [-0.6239573   1.79252393  3.82041516  3.83396102 -1.76897438  0.41282632] met een fitness van 47.2045'

In [5]:
1/fitness

0.016456229195455307

In [6]:
sum( solution * input_values)

43.983543770804545

In [7]:
abs( 44 - sum( solution * input_values))

0.016456229195455307

## Fitness berekenen met neurale netwerken

De fitness berekenen kan realiteit erg complex zijn, daarom wordt meestal een neuraal netwerk gebruikt om dit te bepalen. 

Zo heet een menselijk genoom bestaat uit 3 miljard base paren.

In [3]:
data_inputs = torch.rand(4,2)

data_outputs = torch.tensor([0, 1, 1, 0])

num_inputs = data_inputs.shape[1]

GANN_instance = pygad.gann.GANN(num_solutions=6,            # populatie
                                num_neurons_input=num_inputs,
                                num_neurons_hidden_layers=[2],
                                num_neurons_output=2,
                                hidden_activations=["relu"],
                                output_activation="softmax")

In [51]:
def fitness_func(ga_instance:pygad.GA, solution:torch.Tensor, sol_idx:int):
    predictions = pygad.nn.predict(last_layer=GANN_instance.population_networks[sol_idx],
                                   data_inputs=solution.reshape(data_inputs.shape) )
    correct_predictions = np.where(predictions == data_outputs.numpy())[0].size
    solution_fitness = (correct_predictions/data_outputs.size(0))*100

    return solution_fitness

In [8]:
initial_population = pygad.gann.population_as_vectors(population_networks=GANN_instance.population_networks)

print("Populatie grootte= ", len(initial_population) ,"\nEerste individu =\n", initial_population[0].reshape(4,-1) )

Populatie grootte=  6 
Eerste individu =
 [[-0.03442531  0.00212062]
 [-0.09860268  0.02984727]
 [-0.00598982  0.01744189]
 [-0.00117039  0.0043304 ]]


In [52]:
initial_population = pygad.gann.population_as_vectors(population_networks=GANN_instance.population_networks)

num_parents_mating = 4

num_generations = 500

parent_selection_type = "sss"

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 25  

keep_parents = 1

init_range_low = -2
init_range_high = 5

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       mutation_percent_genes=mutation_percent_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       keep_parents=keep_parents)

In [61]:
ga_instance.run()

In [62]:
solution, fitness, best_fit_idx = ga_instance.best_solution()
print(f"De beste oplossing is\n {solution.reshape(4,-1)}\n met een fitness van {fitness:0.0f}%")

De beste oplossing is
 [[-7.50253134  5.09748509]
 [ 4.41698462  5.83154582]
 [ 5.63148827  6.83744113]
 [-3.90733317  2.34899638]]
 met een fitness van 100%


In [72]:
pygad.nn.predict(last_layer=GANN_instance.population_networks[4],
                   data_inputs= solution.reshape(4,-1) )

[0, 1, 1, 0]